In [1]:
import warnings
warnings.filterwarnings("ignore")

import os, pandas as pd, matplotlib.pyplot as plt, numpy as np, seaborn as sns, scipy.stats as stats, glob
from pynwb import NWBHDF5IO
from pathlib import Path
from convnwb.io import load_nwbfile

## confirm number of subjects, neurons, trials

In [ ]:
# norm_choice = 'shortest_baseline'
norm_choice = 'session_baseline'
df = pd.read_pickle(f'neuron_by_trial_norm_by_{norm_choice}.pkl')

df_encoding = df[df['stim_phase'] == 'encoding']
df_baseline = df[df['stim_phase'] == 'baseline']

df_enc_zeroFR = df_encoding[df_encoding['FR'] == 0] # 7 neurons trials get dropped
df_base_zeroFR = df_baseline[df_baseline['FR'] == 0] # 15 neurons trials get dropped

drop_enc_neur = df_enc_zeroFR['neuron'].unique()
drop_base_neur = df_base_zeroFR['neuron'].unique()
drop_zeroFR_neur = np.concatenate((drop_enc_neur, drop_base_neur))

print(f'neuron trials getting dropped due to having FR=0 in either encoding or baseline {drop_zeroFR_neur}')

# from df_encoding and df_baseline, drop the sessions with 0 FR
df_encoding = df_encoding[~df_encoding['neuron'].isin(drop_zeroFR_neur)]
df_baseline = df_baseline[~df_baseline['neuron'].isin(drop_zeroFR_neur)]

print()
num_subjs, num_sess, num_neurons = len(np.unique(df_encoding['subj'])), len(np.unique(df_encoding['sess'])), len(np.unique(df_encoding['neuron']))
print(f'{num_subjs} subjects, {num_sess} sessions, {num_neurons} neurons')
print(f'number of rows = {len(df_encoding)} = {len(df_baseline)}')

print()
no_fMRI_subjs = ['P49', 'P55', 'P56', 'P57', 'P58']
print(f'subjs who didnt undergo fMRI: {no_fMRI_subjs}')
scene_cells = ['P42CS_R1_47_2_1347_7', 'P42CS_R2_42_3_1096_7', 'P43CS_R1_59_3_1929_1', 'P43CS_R1_76_4_1276_10', 'P43CS_R2_29_1_2303_2',
               'P44CS_R1_15_2_835_8', 'P47CS_R1_10_2_900_8', 'P47CS_R1_18_1_1631_4', 'P47CS_R1_22_4_1379_4', 'P47CS_R1_68_1_964_9',
               'P47CS_R1_74_2_1575_10', 'P47CS_R1_78_1_2233_10', 'P47CS_R2_19_2_3139_4', 'P47CS_R2_20_4_2437_4', 'P47CS_R2_47_1_1478_7',
               'P48CS_R1_9_1_678_8', 'P48CS_R1_21_2_873_4', 'P48CS_R1_30_1_1720_2', 'P48CS_R1_43_2_3181_7', 'P48CS_R2_48_2_936_7',
               'P48CS_R2_58_8_6571_1', 'P49CS_R1_7_1_2581_6', 'P49CS_R2_67_1_1165_10', 'P51CS_R1_48_1_2072_8', 'P51CS_R2_40_3_2542_5',
               'P51CS_R2_80_1_794_9', 'P53CS_R1_10_1_2456_8', 'P53CS_R2_6_4_5480_6', 'P53CS_R2_16_2_6444_8', 'P54CS_R1_17_2_3943_4',
               'P54CS_R1_45_1_2172_7', 'P54CS_R1_54_3_4177_3', 'P54CS_R1_55_2_6861_3', 'P54CS_R1_56_1_1540_3', 'P54CS_R1_70_3_5181_10',
               'P54CS_R2_13_4_7736_8', 'P54CS_R2_67_1_4703_10', 'P55CS_R2_18_2_1050_4', 'P55CS_R2_45_1_1206_7', 'P55CS_R2_48_1_36_7',
               'P56CS_R1_9_1_3423_8', 'P56CS_R1_13_1_999_8', 'P56CS_R1_19_1_1458_4', 'P57CS_R1_66_1_2442_10', 'P57CS_R2_9_1_740_8',
               'P57CS_R2_15_2_1738_8', 'P57CS_R2_70_2_1978_10', 'P58CS_R1_34_3_3299_5', 'P58CS_R1_34_4_3301_5', 'P58CS_R1_35_4_2476_5',
               'P58CS_R1_56_1_3134_3', 'P58CS_R1_57_3_2783_1', 'P58CS_R1_60_1_1424_1', 'P58CS_R1_62_2_3305_1', 'P58CS_R1_63_2_3170_1',
               'P60CS_R1_39_1_1408_5', 'P60CS_R1_54_1_571_3', 'P60CS_R1_56_1_1106_3', 'P62CS_R1_30_3_6240_3', 'P62CS_R2_22_2_2532_7']
print(f'{len(scene_cells)} scene selective cells')

neuron trials getting dropped due to having FR=0 in either encoding or baseline ['P47CS_R1_19_4_1280_4' 'P47CS_R2_22_1_8375_4' 'P48CS_R2_58_3_6420_1'
 'P48CS_R2_62_2_2097_1' 'P51CS_R2_48_4_5337_7' 'P54CS_R2_31_1_2534_2'
 'P55CS_R1_49_2_5469_3' 'P43CS_R2_76_1_990_10' 'P47CS_R1_52_1_298_3'
 'P48CS_R1_24_1_2297_4' 'P48CS_R1_57_1_753_1' 'P48CS_R2_58_3_6420_1'
 'P48CS_R2_62_2_2097_1' 'P49CS_R2_64_1_4257_1' 'P53CS_R1_37_1_3731_5'
 'P54CS_R1_70_2_5145_10' 'P54CS_R2_31_1_2534_2' 'P55CS_R1_54_1_3978_3'
 'P57CS_R2_51_1_624_3' 'P62CS_R1_27_3_8498_3' 'P62CS_R1_29_3_8468_3'
 'P62CS_R1_32_6_6115_3']

16 subjects, 29 sessions, 1438 neurons
number of rows = 1438 = 1438

subjs who didnt undergo fMRI: ['P49', 'P55', 'P56', 'P57', 'P58']
60 scene selective cells
